# Final Assignment

STT: 34

**Member**

18120374 Nguyễn Minh Hiếu


Thư viện cần dùng thêm:
   * **ntlk**: để schemming
   * **auttocorrect**: để kiểm tra chính tả

Lợi ích:
* Nếu tìm được công thức thì công ty sản xuất có thể tự động hóa quá trình rút trích ý kiến thích / không thích từ các đoạn đánh giá của người dùng
* Giúp công ty đánh giá được hiệu quả công việc. (Ở đây là AMAZON có thể kiểm tra sản phẩm được vận chuyển có đúng ý muốn khách hàng hay không)

# Bước 1: Thu thập dữ liệu
Đưa một link sản phẩm bất kì và kiểm tra phần đánh giá của khách hàng.
* Đánh giá sẽ phân loại:*postive* và *critical*
* Mỗi trang gồm 10 đánh giá, ta duyệt 250 trang cho mỗi loại đánh giá. Vậy tổng cộng 10x250x2 = 5000 đánh giá

In [2]:
import requests
import pandas as pd
import numpy as np
import time
import sklearn
from bs4 import BeautifulSoup
from selenium import webdriver
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [ ]:
url = 'https://www.amazon.com/YI-Wireless-Security-Surveillance-Monitor/dp/B01CW4CEMS/ref=sr_1_47?dchild=1&qid=1610065358&s=electronics&sr=1-47'
browser = webdriver.Edge(executable_path='./msedgedriver.exe') 
browser.maximize_window()
browser.get(url)

review_page = browser.find_element_by_link_text('See all reviews').click();

In [ ]:
reviewer_types = browser.find_elements_by_css_selector('#reviewer-type-dropdown option')
for reviewer_type in reviewer_types:
    if "Verified purchase only" in reviewer_type.text:
        reviewer_type.click()
        break

In [ ]:
comment_types = browser.find_elements_by_css_selector('#star-count-dropdown option')
for comment_type in comment_types:
    if "All positive" in comment_type.text:
        comment_type.click()
        break

In [ ]:
phrases = []
status = []

In [ ]:
for i in range(0,260):
    time.sleep(3)
    html_text = browser.page_source
    tree = BeautifulSoup(html_text, 'html.parser')
    reviews = tree.find_all('div',{'data-hook':'review'})
    for review in reviews:
        content = review.find('a', {'data-hook': 'review-title'})
        phrases.append(content.text.strip())
        status.append(1)
    browser.find_element_by_class_name('a-last').click()
   

In [ ]:
comment_types = browser.find_elements_by_css_selector('#star-count-dropdown option')
for comment_type in comment_types:
    if "All critical" in comment_type.text:
        comment_type.click()
        break

In [ ]:
for i in range(0,260):
    time.sleep(3)
    html_text = browser.page_source
    tree = BeautifulSoup(html_text, 'html.parser')
    reviews = tree.find_all('div',{'data-hook':'review'})
    for review in reviews:
        content = review.find('a', {'data-hook': 'review-title'})
        phrases.append(content.text.strip())
        status.append(0)
    browser.find_element_by_class_name('a-last').click()
    

In [ ]:
Data = pd.DataFrame(
    {'Para': phrases,
     'Stt': status
    }
)

**Kiểm tra data**

Lưu data vào file "data.csv"

In [ ]:
Data.shape

In [ ]:
Data.to_csv('data.csv',index = False,encoding='utf-16',sep = '\t')

* cột Para sẽ chứa các đánh giá
* cột status gồm loại đánh giá tương ứng (1: positive - 0: critical)

In [3]:
data = pd.read_csv('data.csv',encoding='utf-16',sep = '\t')
#data.head()
data[data.index == 2458]

,Para,Stt
2458,Great camera,1


In [4]:
data.shape

(5098, 2)

Dữ liệu trùng lặp

In [5]:
data.duplicated().sum()

650

Loại bỏ trùng lặp 

In [6]:
data.drop_duplicates(subset=None, keep='first', inplace=True)
data.shape

(4448, 2)

Kiểm tra thiếu dữ liệu

In [7]:
data.isna().sum()

Para    0
Stt     0
dtype: int64

Kiểm tra độ cân bằng dữ liệu

In [8]:
data.Stt.mean()

0.49280575539568344

In [9]:
y_sr = data["Stt"] # sr là viết tắt của series
X_df = data.drop("Stt", axis=1)

In [10]:
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3, 
                                                              stratify=y_sr, random_state=0)

In [ ]:
train_X_df.shape

In [ ]:
train_y_sr.shape

# Bước 2 tiền xử lý dữ liệu

Ta cần thực hiện các bước: loại phần thừa, áp dụng *stemming* 
* Stemming: đưa các từ có hình thức tương đồng nhau về từ gốc (Ở đây ta chưa bàn về ngữ cảnh).
* Phần thừa như dấu câu, icon.

Vì ta sử dụng tiếng Anh nên các từ có dấu (khác a-z A-Z) cũng sẽ bị loại bỏ

In [17]:
import nltk
import re

#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neural_network import MLPClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import  StandardScaler


In [12]:
englishStemmer=SnowballStemmer("english")
stop_words = set(stopwords.words('english')) 


In [13]:
def Recontruct_Sentence(sentence):
    sentence = re.sub('[^A-Za-z]',' ', sentence)
    sentence = sentence.lower()
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(englishStemmer.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [72]:
#Test pipeline
class trans_Sentence(BaseEstimator, TransformerMixin):
    
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        # YOUR CODE HERE
        optimize= []
        self = X_df.copy()
        sentence_s = self.Para
        for sentence in sentence_s:
            optimize.append(Recontruct_Sentence(sentence))
        self.drop(['Para'],axis = 1)
        self['Para'] = optimize
        print('trans', self.shape)
        return self

In [79]:
class toarray_Tidf(BaseEstimator, TransformerMixin):
    def __init__(self,max_features = None, ngram_range = None, stop_words = None):
        self.max_features = max_features
        self.ngram_range = ngram_range
        self.stop_words = stop_words 
        vect = TfidfVectorizer(max_features =  max_features, ngram_range =  ngram_range, stop_words =  self.stop_words)
        self.vect = vect
    def fit(self, X_df, y=None):
        self.vect.fit(X_df.Para)
        return self
    def transform(self, X_df, y=None):
        # YOUR CODE HERE
        return self.vect.transform(X_df.Para).toarray()

In [80]:
neural = MLPClassifier(hidden_layer_sizes = 10, activation='tanh', solver='lbfgs', random_state=0, max_iter = 2750)
tidf = toarray_Tidf(stop_words = {'english'}, ngram_range = (1,1))
pipe_prog = make_pipeline(trans_Sentence(), tidf, neural)

num_max_features = [500,700,1100,None]
num_alpha = [0.1, 1, 10, 100, 1000]




pipe_prog

Pipeline(steps=[('trans_sentence', trans_Sentence()),
                ('toarray_tidf',
                 toarray_Tidf(ngram_range=(1, 1), stop_words={'english'})),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=10,
                               max_iter=2750, random_state=0,
                               solver='lbfgs'))])

In [81]:
train_errs = []
val_errs = []
best_val_err = float('inf')
best_alpha = None
best_best_feature = None
for max_features in num_max_features:
    for alpha in num_alpha:
        pipe_prog.set_params(toarray_tidf__max_features= max_features, mlpclassifier__alpha=alpha)
        let_fit = pipe_prog.fit(train_X_df,train_y_sr)
        
        error = (1- let_fit.score(train_X_df,train_y_sr))*100
        train_errs.append(error)
        
        error = (1 - let_fit.score(val_X_df,val_y_sr))*100
        val_errs.append(error)
        
        if (error < best_val_err):
            best_val_err = error
            best_alpha = alpha
            best_feature = max_features

trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)
trans (3113, 1)
trans (3113, 1)
trans (1335, 1)


In [82]:
best_val_err

16.25468164794007

In [ ]:
train_y_sr

In [ ]:
result = GridSearchCV(pipe_prog,pipe_para,cv=5,n_jobs=-1)
result


In [ ]:
result.fit(train_X_df, train_y_sr)

In [ ]:
1-result.score(train_X_df, train_y_sr)

In [ ]:
1-result.score(val_X_df,val_y_sr)

In [ ]:
###Test2

In [ ]:
result.best_params_


In [77]:
op = trans_Sentence()
a = op.fit_transform(train_X_df)
vect =  toarray_Tidf(stop_words = {'english'}, ngram_range = (1,1), max_features = 700)
b = vect.fit(a)
c =op.fit_transform(val_X_df)

len(b.transform(a).toarray()[0])


trans (3113, 1)
trans (1335, 1)


700

In [ ]:
# Tạo mô hình Neural Net với: 
# * 1 tầng ẩn gồm 100 nơ-ron, hàm kích hoạt: tanh
# * Thuật toán cực tiểu hóa: LBFGS, số vòng lặp tối đa: 1000
# * random_state: 0
neural_net_model2 = MLPClassifier(hidden_layer_sizes=(500,), activation='tanh',
                                 solver='lbfgs', max_iter=1000, random_state=0)

In [ ]:
neural_net_model2.fit(b, train_y)

In [ ]:
1 - neural_net_model2.score(bag_of_word, train_y_sr)

In [ ]:
op = trans_Sentence()
a = op.fit_transform(test_X_df)
vect =  TfidfVectorizer()
b = vect.fit_transform(a)
b.toarray()


In [ ]:
1 - neural_net_model2.score(bag_of_word, test_y_sr)